In [1]:
sharable_links = ['https://drive.google.com/open?id=1octn7viLBP10qqVkYv5QLaU6RiwpNwFc']
filenames = ['NewestV3BalancedDatasetWithLatentSpaceOutputs.zip']


!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def link2id(link):
  pos = link.find("=")
  return link[pos+1:]

ids = [link2id(link) for link in sharable_links]

for i in range(len(ids)):
  downloaded = drive.CreateFile({'id': ids[i]})
  downloaded.GetContentFile(filenames[i])
  print(filenames[i]," loaded.")

NewestV3BalancedDatasetWithLatentSpaceOutputs.zip  loaded.


In [2]:
!unzip NewestV3BalancedDatasetWithLatentSpaceOutputs.zip

Archive:  NewestV3BalancedDatasetWithLatentSpaceOutputs.zip
   creating: NewestV3BalancedDatasetWithLatentSpaceOutputs/
  inflating: NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedLatentSpaceObj.npy  
  inflating: NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedInputSubImages.npy  
  inflating: NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedOutputPredictions.npy  
  inflating: NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedLatentSpaceSub.npy  
  inflating: NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedInputObjImages.npy  
  inflating: NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedInputObjVectors.npy  
  inflating: NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedInputSubVectors.npy  
  inflating: NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedInputImages.npy  


In [0]:
import numpy as np

bboxImage = np.load("NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedInputImages.npy")
objLatentVector = np.load("NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedLatentSpaceObj.npy")
subLatentVector = np.load("NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedLatentSpaceSub.npy")
objWordVector = np.load("NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedInputObjVectors.npy")
subWordVector = np.load("NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedInputSubVectors.npy")

predOutput = np.load("NewestV3BalancedDatasetWithLatentSpaceOutputs/balancedOutputPredictions.npy")

In [4]:
print(bboxImage.shape)
print(objLatentVector.shape)
print(subLatentVector.shape)
print(objWordVector.shape)
print(subWordVector.shape)
print(predOutput.shape)

(14216, 50, 50, 1)
(14216, 9216)
(14216, 9216)
(14216, 50)
(14216, 50)
(14216, 1)


In [5]:
indices = np.arange(0, bboxImage.shape[0])
filtered_indices = np.zeros(shape=[0])

predicates = ["on", "next to", "behind", "under"]
for i in range(4):
  indices_with_id = indices[predOutput[:,0]==i]
  print(indices_with_id.shape)
  np.random.shuffle(indices_with_id)
  filtered_indices = np.concatenate([filtered_indices, indices_with_id])
print(filtered_indices.shape)
filtered_indices = np.array(filtered_indices, dtype=np.int32)


(6501,)
(3982,)
(1753,)
(1980,)
(14216,)


In [0]:
filteredBBoxImage = bboxImage[filtered_indices]
filteredObjLatentVector = objLatentVector[filtered_indices]
filteredSubLatentVector = subLatentVector[filtered_indices]
filteredObjWordVector = objWordVector[filtered_indices]
filteredSubWordVector = subWordVector[filtered_indices]
filteredPredOutput = predOutput[filtered_indices]

In [7]:
indices = np.arange(0, filteredBBoxImage.shape[0])
chosen_indices = []
total_classes = 0
for i in range(4):
  indices_id = indices[filteredPredOutput[:,0]==i]
  # if indices_id.shape[0]>=threshold:
  chosen_indices.append(i)
  filteredPredOutput[indices_id] = total_classes
  total_classes += 1
print(chosen_indices)

[0, 1, 2, 3]


In [8]:
predicates = ["on", "next to", "behind", "under"]
chosen_preds = []
for i in range(len(chosen_indices)):
  chosen_preds.append(predicates[chosen_indices[i]])

print(chosen_preds)


['on', 'next to', 'behind', 'under']


In [0]:
indices = np.arange(0, filteredBBoxImage.shape[0])
np.random.shuffle(indices)

filteredBBoxImage = filteredBBoxImage[indices]
filteredObjLatentVector = filteredObjLatentVector[indices]
filteredSubLatentVector = filteredSubLatentVector[indices]
filteredObjWordVector = filteredObjWordVector[indices]
filteredSubWordVector = filteredSubWordVector[indices]
filteredPredOutput = filteredPredOutput[indices]

nonOverlappingIndices = indices

In [0]:

test_split = 0.1

test_set_size = int(nonOverlappingIndices.shape[0]*test_split)

def createTestTrainSplit(data, test_set_size=test_set_size):
  return data[test_set_size:], data[:test_set_size]

trainBBoxImage, testBBoxImage = createTestTrainSplit(filteredBBoxImage)
trainObjLatentVector, testObjLatentVector = createTestTrainSplit(filteredObjLatentVector)
trainSubLatentVector, testSubLatentVector = createTestTrainSplit(filteredSubLatentVector)
trainObjWordVector, testObjWordVector = createTestTrainSplit(filteredObjWordVector)
trainSubWordVector, testSubWordVector = createTestTrainSplit(filteredSubWordVector)
trainPred, testPred = createTestTrainSplit(filteredPredOutput)



In [12]:
print(trainBBoxImage.shape)
print(nonOverlappingIndices.shape)

(12795, 50, 50, 1)
(14216,)


In [13]:
import keras
y_train = keras.utils.to_categorical(trainPred, num_classes=total_classes)
y_test = keras.utils.to_categorical(testPred, num_classes=total_classes)

print(np.sum(y_train, axis=0))
print(total_classes)

Using TensorFlow backend.


[5811. 3601. 1592. 1791.]
4


In [0]:
def focal_loss(gamma=2., alpha=4.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed


In [0]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [21]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
import tensorflow as tf

from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten
from keras.models import Model

uniqueCode = "model_"
for i in range(20):
  c = ord('a') + np.random.randint(0, 26, 1)
  uniqueCode += chr(c)
print(uniqueCode)

!mkdir {uniqueCode}

print("Model Name = ",uniqueCode)

inputBBoxImage = Input(shape=(50, 50, 1))
inputObjLatentVector = Input(shape=(9216,))
inputSubLatentVector = Input(shape=(9216,))
inputObjWordVector = Input(shape=(50,))
inputSubWordVector = Input(shape=(50,))



z1 = Conv2D(32, (11, 11), activation='relu', padding='same')(inputBBoxImage)
z2 = Conv2D(32, (11, 11), activation='relu', padding='same')(z1)
z = keras.layers.add([z1, z2])
z1 = Conv2D(32, (11, 11), activation='relu', padding='same')(z)
z2 = Conv2D(32, (11, 11), activation='relu', padding='same')(z1)
z = keras.layers.add([z1, z2])
z = MaxPooling2D((2, 2))(z)
z = Dropout(0.25)(z)

z1 = Conv2D(64, (9, 9), activation='relu', padding='same')(z)
z2 = Conv2D(64, (9, 9), activation='relu', padding='same')(z1)
z = keras.layers.add([z1, z2])
z1 = Conv2D(64, (9, 9), activation='relu', padding='same')(z)
z2 = Conv2D(64, (9, 9), activation='relu', padding='same')(z1)
z = keras.layers.add([z1, z2])
z = MaxPooling2D((2, 2))(z)
z = Dropout(0.25)(z)

z1 = Conv2D(128, (7, 7), activation='relu', padding='same')(z)
z2 = Conv2D(128, (7, 7), activation='relu', padding='same')(z1)
z = keras.layers.add([z1, z2])
z1 = Conv2D(128, (7, 7), activation='relu', padding='same')(z)
z2 = Conv2D(128, (7, 7), activation='relu', padding='same')(z2)
z = keras.layers.add([z1, z2])
z = MaxPooling2D((2, 2))(z)
z = Dropout(0.25)(z)

z = Flatten()(z)
concatenated = keras.layers.concatenate([z, inputObjLatentVector, inputSubLatentVector, inputObjWordVector, inputSubWordVector])
concatenated = Dense(1024, activation='relu')(concatenated)
concatenated = Dense(128, activation='relu')(concatenated)
concatenated = Dropout(0.25)(concatenated)
out = Dense(total_classes, activation='softmax')(concatenated)

model = Model([inputBBoxImage, inputObjLatentVector, inputSubLatentVector, inputObjWordVector, inputSubWordVector], out)

learning_rate = 0.00003

adam = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy', precision_m, recall_m, f1_m])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
filepath = "best_model.h5"
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)
batch_size = 64
epochs = 400

model.fit([trainBBoxImage, trainObjLatentVector, trainSubLatentVector, trainObjWordVector, trainSubWordVector], [y_train], verbose=1,
            epochs=2, batch_size=batch_size, shuffle=True, validation_split=0.1, callbacks=[reduce_lr, model_checkpoint])

learning_rate2 = 1.73205e-5
adam2 = Adam(lr=learning_rate2, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimizer=adam2, loss=focal_loss(alpha=1), metrics=['accuracy', precision_m, recall_m, f1_m])
model.fit([trainBBoxImage, trainObjLatentVector, trainSubLatentVector, trainObjWordVector, trainSubWordVector], [y_train], verbose=1,
            epochs=18, batch_size=batch_size, shuffle=True, validation_split=0.1, callbacks=[reduce_lr, model_checkpoint])

learning_rate3 = 7.59835e-6
adam3 = Adam(lr=learning_rate3, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimizer=adam3, loss=focal_loss(alpha=1), metrics=['accuracy', precision_m, recall_m, f1_m])
model.fit([trainBBoxImage, trainObjLatentVector, trainSubLatentVector, trainObjWordVector, trainSubWordVector], [y_train], verbose=1,
            epochs=epochs-20, batch_size=batch_size, shuffle=True, validation_split=0.1, callbacks=[reduce_lr, model_checkpoint])



model_vvvrpwhxwavgcfeawika
Model Name =  model_vvvrpwhxwavgcfeawika
Train on 11515 samples, validate on 1280 samples
Epoch 1/2
11515/11515 [==============================] - 14s 1ms/step - loss: 1.3834 - acc: 0.5153 - precision_m: 0.6130 - recall_m: 0.3337 - f1_m: 0.4271 - val_loss: 0.9603 - val_acc: 0.6125 - val_precision_m: 0.6957 - val_recall_m: 0.4734 - val_f1_m: 0.5627
Epoch 2/2
11515/11515 [==============================] - 12s 1ms/step - loss: 0.8460 - acc: 0.6621 - precision_m: 0.7565 - recall_m: 0.5316 - f1_m: 0.6231 - val_loss: 0.9024 - val_acc: 0.6445 - val_precision_m: 0.7152 - val_recall_m: 0.5508 - val_f1_m: 0.6216
Train on 11515 samples, validate on 1280 samples
Epoch 1/18
11515/11515 [==============================] - 14s 1ms/step - loss: 0.2688 - acc: 0.7720 - precision_m: 0.8852 - recall_m: 0.6066 - f1_m: 0.7185 - val_loss: 0.4606 - val_acc: 0.6492 - val_precision_m: 0.7567 - val_recall_m: 0.4852 - val_f1_m: 0.5894
Epoch 2/18
11515/11515 [=============================

KeyboardInterrupt: ignored

In [22]:
performance = model.evaluate(x=[testBBoxImage, testObjLatentVector, testSubLatentVector, testObjWordVector, testSubWordVector], y=[y_test], batch_size=batch_size, verbose=0)
print(performance)

[0.7360104539711487, 0.6903589024332356, 0.7070171431199839, 0.6586910628836227, 0.6817479650794747]
